In [1]:
import pandas as pd
import numpy as np
from causalimpact import CausalImpact
import csv
import sys
from datetime import datetime
import dtw
from sklearn import preprocessing

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



principal variables:
test set
up to 5 initial like-for-like csv datasets
metric to aggregate by in a str
rolling window
min number of time periods in markets (find best recommended)
min number of time periods in test set (find best recommended)

optional:
queries to be excuded in a list (possibly)
countries
outlier cutoff

returns a csv ready to find matching markets

actions on transform():
check column names
get test_set, test it is not empty
groupby, sum/mean
filter by test_set
rename test markets to 'test'
groupby, sum/mean
    returns:
    one or many DataFrames if 'queries' or 'countries' non empty

In [16]:
pdp = pd.read_excel('C:/Users/akiro/Documents/Adidas/USA/us_test_06_pdp_titles_categories/data/inputs/pdps.xlsx')

In [17]:
data1 = pd.read_csv('C:/Users/akiro/Documents/Adidas/USA/us_test_06_pdp_titles_categories/data/inputs/us_2021_11.csv')
data2 = pd.read_csv('C:/Users/akiro/Documents/Adidas/USA/us_test_06_pdp_titles_categories/data/inputs/bq-all.csv')
data1 = data1.groupby(['date','page']).sum().reset_index()
data1 = data1[['date', 'page', 'impressions']]
data2 = data2[['date', 'page', 'impressions']]

In [18]:
data = pd.concat([data1,data2], ignore_index=True).sort_values('date')

In [54]:
def distance(testset=pd.DataFrame(), dataset=pd.DataFrame(), metric='impressions', ranks='date',
              col='page', roll=2, outlier=1, scaling=True, min_test=320, min_data=300, metric2='sum'):

    testset.columns = testset.columns.str.lower()
    dataset.columns = dataset.columns.str.lower()
    col = col.lower()
    
    assert testset[col].name == testset[col].name, 'Column names must match for the matched markets'
    
    if metric2 == 'sum':
        causal_input = dataset.groupby([ranks, col]).sum().sort_values(ranks, ascending=False).reset_index()
    elif metric2 == 'mean':
        causal_input = causal_input.groupby([ranks, col]).mean().sort_values(ranks, ascending=False).reset_index()
    else:
        raise ValueError('Supported aggregators are sum and mean')

    test_set = causal_input[causal_input[col].isin(pdp[col])]
    
    assert (test_set.iloc[1,:].all() != None)  & (test_set.iloc[0,:].all() != None), 'No markets match on test and data, check strings'

    testcount = test_set.groupby([col]).count().sort_values(metric, ascending=False).reset_index()
    
    assert testcount[metric].max() >= min_test, 'min_test must be lower than the maximum number of observations in testset'
    
    test_tops = testcount[testcount[ranks] >= min_test].reset_index()
    
    if metric2 == 'sum':
        test_set =  causal_input[causal_input[col].isin(test_tops[col])]
        test_set = test_set.groupby([ranks, col]).sum().sort_values(ranks, ascending=False).reset_index()
    elif metric2 == 'mean':
        test_set =  causal_input[causal_input[col].isin(test_tops[col])]
        test_set = test_set.groupby([ranks, col]).mean().sort_values(ranks, ascending=False).reset_index()
        
    if outlier != 1:
        test_set_clean = pd.DataFrame()
        for i in test_tops[col].unique():
            cutoff = test_set[test_set[col] == i].quantile(outlier)
            temp = test_set[test_set[col] == i]
            temp = temp[temp[metric] < cutoff[0]]
            test_set_clean = pd.concat([test_set_clean, temp], ignore_index=True)
            test_set = test_set_clean

        causal_input = causal_input[~causal_input[col].isin(testcount[col])]
        causal_input = pd.concat([causal_input, test_set], ignore_index=True).sort_values(ranks)
    
    causal_input.loc[causal_input[col].isin(test_tops[col]), col] = 'TEST'
    
    if metric2 == 'sum':
        causal_input = causal_input.groupby([ranks, col]).sum().sort_values(ranks, ascending=True).reset_index()
    elif metric2 == 'mean':
        causal_input = causal_input.groupby([ranks, col]).mean().sort_values(ranks, ascending=True).reset_index()
    else:
        raise ValueError('Supported aggregators are sum and mean')
    
    marketcount = causal_input.groupby([col]).count().sort_values(metric, ascending=False).reset_index()

    if marketcount[marketcount[col] == 'TEST'][metric].max() <= min_data:
        min_data = marketcount[marketcount[col] == 'TEST'][metric].max()
    
    assert marketcount[metric].max() >= min_data, 'min_data must be lower than the maximum number of observations in dataset'    
    assert min_test >= min_data, 'Test observations have to be equal or higher than cutoff point'
    
    control_urls = marketcount[marketcount[ranks] >= min_data].reset_index()

    causal_control = causal_input.loc[causal_input[col].isin(control_urls[col]),]

    pvt_table = causal_control.pivot_table(index=ranks, columns=col, values=metric, aggfunc=metric2).reset_index().fillna(0).set_index(ranks)

    pvt_table = pvt_table.rolling(roll).mean()

    pvt_table = pvt_table[roll-1:]

    causal_control = pvt_table.melt(ignore_index=False).reset_index().sort_values(ranks).reset_index(drop=True)

    markets = {}
    for i in causal_control[col].unique():
        markets[i] = causal_control[causal_control[col] == i].sort_values(ranks).reset_index(drop=True)[['value']]
    
    distances = {}
    for i in causal_control[col].unique():
        distances[i] = dtw.dtw(markets['TEST'], markets[i]).distance
        
    final = pd.DataFrame.from_dict(distances, orient='index', columns=['dist']).sort_values('dist', ascending=True)[1:].reset_index()

    if scaling == True:
        x = final.dist[:].values
        min_max_scaler = preprocessing.MaxAbsScaler()
        x_scaled = min_max_scaler.fit_transform(x.reshape(-1, 1))
        final.dist = x_scaled
    
    return  final

In [57]:
%%time
temp = distance(testset = pdp,
               dataset= data,
               metric= 'impressions' ,
               col= 'page',
                metric2 = 'sum',
                min_test=320,
                min_data=300,
                outlier=0.90
               )

Wall time: 10 s
